In [1]:
import numpy as np
import math
import pandas as pd

In [2]:
def GenAdjMat(p, N):
    x = []
    for i in range(0,N**2):
        xcor = math.ceil((i+1)/N)
        ycor = (i+1) - (N * math.floor(((i+1)/N) - 0.000001))
        if xcor == ycor:
            x.append(0)
        if xcor != ycor:
            relv = int((N* (ycor - 1)) + xcor)
            if(relv < (i+1)):
                x.append(x[relv-1])
            else:
                d = np.random.uniform(0,1)
                if d <= p:
                    x.append(1)
                else:
                    x.append(0)
            
    x = np.array(x)
    x = x.reshape(N,N)
    
    return x

In [3]:
def GenNOLMat(x):
    y = []

    for i in x:
        for j in x:
            a = i + j
            cf = 0
            tf = 0
            for k in a:
                if k >= 1:
                    tf += 1
                if k == 2:
                    cf += 1
            tf -= 2
            if tf != 0:
                y.append(cf/tf)
            else:
                y.append(0)

    y = np.array(y)
    y = y.reshape(np.shape(x)[1], np.shape(x)[1])
    for i in range(0, np.shape(x)[1]):
        for j in range(0, np.shape(x)[1]):
            if x[i][j] == 0:
                y[i][j] = 0
    
    return y

In [4]:
def GenDCenty(y):
    difcen = [0] * ((np.shape(y)[1])**2)
    for i in range(0, (np.shape(y)[1])**2):
        difcen[i] = float(difcen[i])
    
    difcen = np.array(difcen)
    difcen = difcen.reshape(np.shape(y)[1], np.shape(y)[1])
    
    for i in range(0,10):
        a = y**i
        difcen += a
    
    difcenf = []

    for i in range(0, np.shape(y)[1]):
        difcenf.append(sum(difcen[i]))
    
    return difcenf

In [5]:
def GenNodList(adjmat, nolmat, dcvect, vtrus):
    nodlist = []
    
    for i in range(0, np.shape(nolmat)[1]):
        vect = [nolmat[i], 0, vtrus, i]
        nodlist.append(vect)
    
    return nodlist

In [6]:
a = [[1,2],[3,4],[5,6],[7,8],[9,10]]
print(np.shape(a)[0])

5


In [7]:
def RunSimu(nodlist, time):
    for t in range(1, time):
        for i in range(1,(np.shape(nodlist)[0])):
            for j in range(0, len(nodlist[i][0])):
                rnd = np.random.uniform(0,1,1)
                if rnd <= nodlist[i][0][j] and nodlist[j][1] == 1:
                    nodlist[i][2] = nodlist[i][2] + (nodlist[i][0][j] * (1-nodlist[i][2]))
        for i in nodlist:
            if max(i[0]) != 0:
                i[1] = 1
            if max(i[0]) == 0:
                i[1] = 0
    vtrusts = []
    for i in nodlist:
        vtrusts.append(i[2])
    success = np.mean(vtrusts)
    
    return success

In [8]:
columns = ['serial', 'density', 'duration', 'initial trust', 'number of seeds', 'average trust']
data = []

In [9]:
for iteration in range(0,200):
    dur = np.random.randint(5, 30)
    vtrust1 = np.random.uniform(0, 1)
    seednum = np.random.randint(1, 10)
    density = np.random.uniform(0.1, 1)
    
    a = GenAdjMat(density, 100)
    b = GenNOLMat(a)
    c = GenDCenty(b)
    d = GenNodList(a, b, c, vtrust1)
    
    for trial in range(1, seednum):
        mxvalue = max(c)
        x = c.index(mxvalue)
        d[x][1] = 1
        c[x]=-1000
        
    
    avtrust = RunSimu(d, dur)
        
    data.append([iteration, density, dur, vtrust1, seednum, avtrust])
    
    if np.floor(iteration/10) == iteration/10:
        print(iteration, "out of", 1000)

/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


0 out of 1000
10 out of 1000
20 out of 1000
30 out of 1000
40 out of 1000
50 out of 1000
60 out of 1000
70 out of 1000
80 out of 1000
90 out of 1000
100 out of 1000
110 out of 1000
120 out of 1000
130 out of 1000
140 out of 1000
150 out of 1000
160 out of 1000
170 out of 1000
180 out of 1000
190 out of 1000


In [10]:
df = pd.DataFrame(data = data,
                 columns = columns)

print(df)

     serial   density  duration  initial trust  number of seeds  average trust
0         0  0.639843        14       0.485161                8       0.994852
1         1  0.100803        21       0.275270                3       0.650746
2         2  0.509620         6       0.451487                2       0.994515
3         3  0.839383         8       0.502913                2       0.995029
4         4  0.640023         6       0.101238                6       0.991012
..      ...       ...       ...            ...              ...            ...
195     195  0.690168        14       0.133002                1       0.991330
196     196  0.551971        27       0.867087                1       0.998671
197     197  0.637814        18       0.966817                5       0.999668
198     198  0.968803        23       0.199755                7       0.991998
199     199  0.342273        14       0.279467                3       0.992776

[200 rows x 6 columns]


In [11]:
pip install openpyxl

     |████████████████████████████████| 243 kB 4.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
with pd.ExcelWriter('testtreatment1.xlsx') as writer:
    df.to_excel(writer)

In [15]:
print(c)

[175.60206772181053, 159.22824344370815, 163.14056184442313, 158.40490508261757, 159.72873484111489, 165.02042294807623, 169.66516835203976, 166.85698040168705, 157.38419131462436, 160.37447510477529, 172.84255023437626, 179.132169352045, 179.1853971519024, 149.8002483192929, 160.27677774360924, 158.16075895265567, 167.58154534351158, 162.4424987899063, 150.26230449632084, 160.81802767116076, 146.2693707706881, 165.77047676594694, 156.16212854486645, 186.12320503324506, 142.21438490011602, 143.32873344442797, 162.57883721211897, 168.32994837533883, 178.62236094836732, 156.74223458777107, 156.81587283642696, 161.01327356857684, 138.37922723853364, 138.44466603263663, 164.77071593455216, 138.30484457419323, 170.23327300321114, 173.79112597808614, 172.48060700117972, 172.32693870056332, 175.3879349382889, 151.2567207958923, 174.9819260097155, 162.95147595765266, 171.8845483883477, 168.00906664443227, 171.50933162057706, 162.63480343316527, 149.31305818197401, 166.73177145959377, 164.98393